In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")


if openai_api_key:
    print("OPENAI_API_KEY Present: ", openai_api_key[:8])

if groq_api_key:
    print("GROQ_API_KEY Present: ", groq_api_key[:8])

OPENAI_API_KEY Present:  sk-proj-
GROQ_API_KEY Present:  gsk_LGQu


In [4]:
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [5]:
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

In [6]:
groq_model = OpenAIChatCompletionsModel(model='llama-3.3-70b-versatile', openai_client=groq_client)

In [7]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

sales_agent1 = Agent(
    name="Groq Sales Agent 1",
    instructions=instructions1,
    model=groq_model
)

sales_agent2 = Agent(
    name="Groq Sales Agent 2",
    instructions=instructions1,
    model=groq_model
)
sales_agent3 = Agent(
    name="Groq Sales Agent 3",
    instructions=instructions1,
    model=groq_model
)

In [8]:
description="Write a cold sales email."

tool1 = sales_agent1.as_tool(tool_name='sales_agent1', tool_description=description)
tool2 = sales_agent2.as_tool(tool_name='sales_agent2', tool_description=description)
tool3 = sales_agent3.as_tool(tool_name='sales_agent3', tool_description=description)

In [9]:
@function_tool
def send_html_mail(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and the given body to all sales prospects."""

    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("jai.300012723027@csvtu.ac.in")
    to_email = To("ksharma9719@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {'status': 'success'}

In [10]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(
    name = "Subject Writer",
    instructions=subject_instructions,
    model = 'gpt-4.1-nano'
)
subject_tool = subject_writer.as_tool(tool_name = "subject_writer", tool_description="Write a subject for a cold sales email.")


html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4.1-nano")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [11]:

email_tools = [subject_tool, html_tool, send_html_mail]

In [12]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Emailer Agent",
    tools = email_tools,
    model = 'gpt-4o-mini',
    instructions=instructions,
    handoff_description="Convert an Email to HTML and send it."
)

In [ ]:
tools = [tool1, tool2, tool3]

sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools = tools,
    handoffs = [emailer_agent],
    model = 'gpt-4o-mini'
)

message = "send a cold sales email to Dear CEO from Jai Keshav Sharma"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

Check out the traces: https://platform.openai.com/traces

### **Introducing Guardrails**

Guardrails can either be applied to input or output, not anywhere else.

In [26]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [27]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"  # will trigger the tripwire 

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [29]:
# won't trigger the tripwire

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

Check out the trace: https://platform.openai.com/traces